In [12]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import load_img, img_to_array, array_to_img, to_categorical
import tensorflow.keras.backend as K

## Dice Loss

### Data synthesis

This is false synthesis of ground truth and predicion data.
- The ground truth is data that is sampled from a uniform distribution with the minimum of 0 and the maximum of 2, corresponding to 3 classes. These values are then encoded in one-hot form
- The predicion data is data is also sampled from a uniform distribution with the minimum of 0 and the maximum of 2 however it's sampled with 3 dimensions to produce replicate probability scores, then the softmax function is applied to the data to convert it into bounded probabilities.

Moreover, the data is such that it is batched as 32 synthetic images

In [262]:
y_true = np.round(np.random.uniform(0, 2, (32, 224, 224, 1)))
y_true = tf.one_hot(y_true, depth=3, axis=-1)
y_true = tf.squeeze(y_true)
y_pred = np.random.uniform(0, 10, (32, 224, 224, 3))
#y_pred = np.zeros((224,224,3))
y_pred = tf.nn.softmax(y_pred)
y_pred = tf.cast(y_pred, tf.float32)
#y_pred = tf.one_hot(y_pred, depth=3, axis=-1)
#y_pred = tf.squeeze(y_pred)

### Manually synthesized data

This data has been manually synthesized rather than being sampled from a distribution.

In [220]:
y_pred = np.array([[[0.05, 0.9, 0.05]]])
y_pred = np.repeat(y_pred, (224), axis=1)
y_pred = np.repeat(y_pred, (224), axis=0)
y_pred.shape

(224, 224, 3)

In [221]:
y_pred[0,0]

array([0.05, 0.9 , 0.05])

### Computing the dice loss between just 2 images

In [254]:
#Implementation with just single image
numerator = y_true[0] * y_pred[0] # Broadcasting
numerator = tf.reduce_sum(numerator, axis=0)
numerator = 2 * tf.reduce_sum(numerator, axis=0)
denominator = (y_true[0] + y_pred[0])# Broadcasting
denominator = tf.reduce_sum(denominator, axis=0)
denominator = tf.reduce_sum(denominator, axis=0)

frac = numerator / denominator
dice_coef = tf.reduce_mean(frac)


loss = 1 - dice_coef

In [256]:
loss

<tf.Tensor: shape=(), dtype=float32, numpy=0.6767421>

### Dice loss computation by batch

In [259]:
#Implementation with batched data
numerator = y_true * y_pred # Broadcasting
numerator = tf.reduce_sum(numerator, axis=1)
numerator = 2 * tf.reduce_sum(numerator, axis=1)
denominator = (y_true + y_pred)# Broadcasting
denominator = tf.reduce_sum(denominator, axis=1)
denominator = tf.reduce_sum(denominator, axis=1)

frac = numerator / denominator
frac = tf.reduce_mean(frac, axis=1)

dice_coef = frac
loss = 1 - dice_coef

In [261]:
loss

<tf.Tensor: shape=(32,), dtype=float32, numpy=
array([0.6767421 , 0.67647696, 0.67619085, 0.67637753, 0.675279  ,
       0.67679393, 0.6769104 , 0.6761048 , 0.67713714, 0.67621017,
       0.6761075 , 0.67689705, 0.67631686, 0.67643476, 0.67648935,
       0.6768576 , 0.6771984 , 0.6760597 , 0.67442226, 0.6759293 ,
       0.6770885 , 0.6757326 , 0.6749605 , 0.674934  , 0.6753901 ,
       0.6757736 , 0.6767249 , 0.67697144, 0.67581034, 0.676671  ,
       0.67657244, 0.676347  ], dtype=float32)>

In [230]:
numerator = y_true * y_pred # Broadcasting
numerator = 2. * tf.reduce_sum(numerator)

denominator = (y_true + y_pred)# Broadcasting
denominator = tf.reduce_sum(denominator)

loss = 1 - numerator / denominator

In [232]:
loss

<tf.Tensor: shape=(), dtype=float32, numpy=-71.51459>

In [134]:
(1 - numerator / denominator)

<tf.Tensor: shape=(), dtype=float32, numpy=0.5>

## Experiments with Categorical Crossentropy

In [87]:
loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_true, y_pred, axis = 2))

In [88]:
loss

<tf.Tensor: shape=(), dtype=float32, numpy=1.2052753>

In [105]:
ce = -1 * tf.reduce_sum(y_true * np.log(y_pred), axis=2)

In [108]:
ce = tf.reduce_mean(ce)

In [109]:
ce

<tf.Tensor: shape=(), dtype=float32, numpy=1.2068808>